<a href="https://colab.research.google.com/github/SemihDemir13/chatbot-project-data/blob/main/V7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Tüm kütüphaneler (Eğitim + RAG) kuruluyor...")
!pip install -q -U transformers huggingface_hub accelerate sentence-transformers torch langchain langchain-community langchain-huggingface chromadb pandas peft trl bitsandbytes

import warnings
import pandas as pd
import torch
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
from datasets import Dataset

warnings.filterwarnings('ignore')
print("✅ Kurulum tamamlandı.")

In [ ]:
from huggingface_hub import login
login()

In [ ]:
# --- ADIM 1: VERİ HAZIRLIĞI (Hafifletilmiş) ---
print("📊 Veri seti eğitim için hazırlanıyor...")
try:
    df_raw = pd.read_csv('netflix_titles.csv')
    df_raw = df_raw.fillna('')

    # Sadece 100 satırlık bir eğitim verisi alıyoruz
    train_df = df_raw.head(100)

    # Modeli eğiteceğimiz format
    def format_row(row):
        return f"Movie: {row['title']}\nType: {row['type']}\nDescription: {row['description']}"

    # Pandas'tan HuggingFace Dataset formatına çevir
    dataset = Dataset.from_pandas(train_df)
    dataset = dataset.map(lambda x: {"text": format_row(x)})

    print("✅ Mini eğitim seti hazırlandı.")

except Exception as e:
    print(f"Hata: CSV okunurken sorun oldu. {e}")

# --- ADIM 2: MODELİ EĞİTİM MODUNDA YÜKLEME ---
print("\n🧠 Model 4-bit (Eğitim Modu) olarak yükleniyor...")
model_id = "google/gemma-2b-it"

# 4-Bit ayarları
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# --- ADIM 3: FINE-TUNING (LoRA Ayarları) ---
print("\n🏋️‍♂️ Fine-Tuning (Eğitim) Başlıyor... (Bu işlem 2-3 dk sürebilir)")

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

training_args = TrainingArguments(
    output_dir="./sonuclar",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    max_steps=30,
    optim="paged_adamw_8bit"
)

# DÜZELTME BURADA YAPILDI: dataset_text_field parametresi kaldırıldı.
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_args
)

# NOT: Eğer yukarıdaki kod yine aynı hatayı verirse, 'dataset_text_field="text",' satırını silip tekrar çalıştır.
# Bazı versiyonlarda gerekmiyor. Ben şimdilik bıraktım ama hata devam ederse silmen gerekebilir.
# GÜNCELLEME: Hata almamak için garanti yöntem o satırı silmektir. Aşağıda silinmiş haliyle trainer'ı tekrar tanımlıyorum:

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_args
)
# (dataset içinde zaten "text" sütunu olduğu için otomatik tanıyacaktır)

trainer.train()
print("✅ Fine-Tuning tamamlandı! Model artık Netflix verilerine aşina.")

# --- ADIM 4: EĞİTİLMİŞ MODELİ RAG İÇİN HAZIRLAMA ---
print("\n🚀 Chatbot Pipeline oluşturuluyor...")

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=250
)
print("✅ Sistem kullanıma hazır!")

In [ ]:
#4
import pandas as pd
import sys

print("CSV dosyası okunuyor ve veri formatlanıyor...")

try:
    df = pd.read_csv('netflix_titles.csv')
    df = df.fillna('Bilinmiyor')

    dokumanlar = []

    # Tüm veriyi alıyoruz
    print(f"Toplam {len(df)} adet kayıt işleniyor...")

    for index, row in df.iterrows():
        # Formatı sadeleştirdik. Başlık en üstte.
        metin = (
            f"Başlık: {row['title']}\n"
            f"Yönetmen: {row['director']}\n"
            f"Yıl: {row['release_year']}\n"
            f"Tür: {row['listed_in']}\n"
            f"Özet: {row['description']}\n"
            f"Detay: Bu film {row['release_year']} yılında yayınlanmıştır. {row['type']} kategorisindedir."
        )
        dokumanlar.append(metin)

    print(f"✅ {len(dokumanlar)} adet içerik başarıyla hazırlandı.")

except FileNotFoundError:
    print("❌ HATA: Dosya yok.")

In [ ]:
#5
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

print("Vektör veritabanı (Çok Dilli) oluşturuluyor...")

# DEĞİŞİKLİK BURADA:
# Artık Türkçe-İngilizce eşleştirmesi yapabilen 'multilingual' modeli kullanıyoruz.
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# ChromaDB veritabanını kur
# Eğer hata alırsan önceki veritabanını temizlemek gerekebilir ama genellikle üzerine yazar.
vektor_veritabani = Chroma.from_texts(dokumanlar, embedding=embedding_model)

print("✅ Çok dilli veritabanı hazır! Artık Türkçe sorguları İngilizce verilerle eşleştirebilirim.")

In [ ]:
import re
import random

def temizle_yorumlar(metin):
    """Modelin sonuna eklediği İngilizce yorumları ve bozuk kısımları siler."""
    yasakli_cumleler = [
        "I hope this", "Please note", "This phrase", "The phrase means",
        "This is a", "Translated", "Here is", "What does", "English:", "Turkish:"
    ]
    satirlar = metin.split('\n')
    temiz_satirlar = []
    for satir in satirlar:
        if any(yasak in satir for yasak in yasakli_cumleler):
            break
        if satir.strip():
            temiz_satirlar.append(satir)
    return " ".join(temiz_satirlar).strip()

def rag_yanit(soru, top_k=3):
    # --- HAZIRLIK ---
    tur_ceviri = {
        "korku": "Horror", "gerilim": "Thriller", "aksiyon": "Action",
        "komedi": "Comed", "drama": "Dram", "romantik": "Romantic", "aşk": "Romantic",
        "bilim kurgu": "Sci-Fi", "belgesel": "Documentary", "çocuk": "Children",
        "aile": "Family", "anime": "Anime", "macera": "Adventure", "suç": "Crime"
    }

    etkisiz_kelimeler = ["filmi", "film", "öner", "bana", "hakkında", "olan", "yapımı", "yönetmen", "oyuncu", "kimdir", "listesi", "var", "mı", "mu", "bir", "söyle", "istiyorum", "tavsiye", "et", "izle"]

    temiz_soru = soru.lower()
    for kelime in etkisiz_kelimeler:
        temiz_soru = temiz_soru.replace(f" {kelime} ", " ").replace(f" {kelime}", "").replace(f"{kelime} ", "")

    anahtar_kelime = temiz_soru.strip()
    anahtar_kelimeler_listesi = anahtar_kelime.split()

    secilen_film_bilgisi = {}
    kaynak_yontem = ""
    uygun_filmler = pd.DataFrame()

    # --- 1. ADIM: ANALİZ ---
    yil_tespiti = re.search(r'(\d{4})', soru)
    min_yil = int(df['release_year'].min())
    max_yil = int(df['release_year'].max())

    # Tür Tespiti
    aranan_tur_eng = None
    for tr_tur, eng_tur in tur_ceviri.items():
        if tr_tur in soru.lower():
            aranan_tur_eng = eng_tur
            break

    # --- 2. ADIM: ARAMA SENARYOLARI ---

    # SENARYO A: YIL VARSA
    if yil_tespiti:
        aranan_yil = int(yil_tespiti.group(1))
        if aranan_yil < min_yil or aranan_yil > max_yil:
            return f"Üzgünüm, veri setimde sadece {min_yil} ile {max_yil} yılları arasındaki filmler bulunmaktadır. ({aranan_yil} mevcut değil)"

        kaynak_yontem = f"YIL FİLTRESİ ({aranan_yil})"
        uygun_filmler = df[df['release_year'] == aranan_yil]

        # Yılın içinde Tür de varsa süz
        if aranan_tur_eng and not uygun_filmler.empty:
            tur_filtreli = uygun_filmler[uygun_filmler['listed_in'].str.contains(aranan_tur_eng, case=False, na=False)]
            if not tur_filtreli.empty:
                uygun_filmler = tur_filtreli
                kaynak_yontem += f" + TÜR ({aranan_tur_eng})"

    # SENARYO B: SADECE TÜR VARSA (YENİ EKLENDİ!)
    # Eğer yıl yoksa ama kullanıcı "Korku filmi" dediyse buraya girecek.
    elif aranan_tur_eng:
        kaynak_yontem = f"TÜR FİLTRESİ ({aranan_tur_eng})"
        uygun_filmler = df[df['listed_in'].str.contains(aranan_tur_eng, case=False, na=False)]

    # SENARYO C: İSİM VARSA (Yıl ve Tür yoksa isimdir)
    elif len(anahtar_kelime) > 2:
        temp_df_cast = df.copy()
        temp_df_dir = df.copy()
        for kelime in anahtar_kelimeler_listesi:
            temp_df_cast = temp_df_cast[temp_df_cast['cast'].str.contains(kelime, case=False, na=False)]
            temp_df_dir = temp_df_dir[temp_df_dir['director'].str.contains(kelime, case=False, na=False)]

        uygun_filmler = pd.concat([temp_df_cast, temp_df_dir]).drop_duplicates()
        if not uygun_filmler.empty:
            kaynak_yontem = f"İSİM FİLTRESİ ('{anahtar_kelime}')"

    # --- 3. ADIM: VERİYİ SEÇME ---
    if not uygun_filmler.empty:
        secilen = uygun_filmler.sample(n=1).iloc[0]
        secilen_film_bilgisi = {
            "title": secilen['title'],
            "year": secilen['release_year'],
            "desc": secilen['description'],
            "genre": secilen['listed_in']
        }
        print(f"\n[SİSTEM: {kaynak_yontem} bulundu: {secilen['title']}]")

    # SENARYO D: VEKTÖR ARAMASI (Son Çare)
    else:
        if yil_tespiti: return f"Aradığınız kriterlere uygun film bulunamadı."

        kaynak_yontem = "VEKTÖR ARAMASI"
        docs = vektor_veritabani.similarity_search(soru, k=1)
        if docs:
            doc = docs[0]
            lines = doc.page_content.split('\n')
            title, desc, year, genre = "Bilinmiyor", "Açıklama yok.", "???", "???"
            for line in lines:
                if "Başlık:" in line: title = line.replace("Başlık: ", "").strip()
                if "Özet:" in line or "Description:" in line: desc = line.split(":", 1)[1].strip()
                if "Yıl:" in line: year = line.replace("Yıl: ", "").strip()
                if "Tür:" in line: genre = line.replace("Tür: ", "").strip()

            secilen_film_bilgisi = {"title": title, "year": year, "desc": desc, "genre": genre}
            print(f"\n[SİSTEM: Vektörden bulundu: {title}]")

    print("-" * 40)

    # --- 4. ADIM: ÇEVİRİ ---
    if not secilen_film_bilgisi: return "Üzgünüm, kriterlerinize uygun bir film bulamadım."

    ingilizce_ozet = secilen_film_bilgisi['desc']
    prompt = f"Task: Translate the text below into fluent Turkish. Do not explain. Text: {ingilizce_ozet}\nTranslation:"

    try:
        sonuc = llm_pipeline(prompt, return_full_text=False, max_new_tokens=100)[0]["generated_text"]
        turkce_ozet = temizle_yorumlar(sonuc.strip())
        if len(turkce_ozet) < 5: turkce_ozet = ingilizce_ozet
    except:
        turkce_ozet = ingilizce_ozet

    # --- 5. ADIM: CEVAP ---
    final_cevap = (f"Önerim: {secilen_film_bilgisi['title']} ({secilen_film_bilgisi['year']})\n"
                   f"Tür: {secilen_film_bilgisi['genre']}\n"
                   f"Konusu: {turkce_ozet}")

    return final_cevap


In [ ]:

print("\n================================================")
print("🎬 NETFLIX GURUSU (Çıkmak için 'çıkış' yazın)")
print("================================================\n")

while True:
    soru = input("Soru sor: ")
    if soru.lower() in ['çıkış', 'exit', 'quit']:
        print("İyi seyirler 👋")
        break

    print("🔎 Araştırıyorum...")
    cevap = rag_yanit(soru)
    print(f"\n🤖 AI: {cevap}\n")
    print("-" * 40)

In [ ]:
!pip install -q streamlit
!npm install localtunnel

In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import torch
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
import warnings

# --- SAYFA AYARLARI ---
st.set_page_config(page_title="Netflix AI Guru", page_icon="🎬", layout="centered")
st.title("🎬 Netflix AI Bot")
st.markdown("*Yapay Zeka Destekli Film & Dizi Öneri Asistanı*")

# --- 1. MODEL VE VERİ YÜKLEME (CACHE) ---
# Streamlit her tıklamada kodu baştan çalıştırır.
# Modeli tekrar tekrar yüklemesin diye @st.cache kullanıyoruz.

@st.cache_resource
def sistemi_yukle():
    print("Sistem yükleniyor...")

    # 1. Veri Seti
    try:
        df = pd.read_csv('netflix_titles.csv')
        df = df.fillna('')
    except:
        st.error("CSV dosyası bulunamadı! Lütfen netflix_titles.csv dosyasını yükleyin.")
        return None, None, None, None

    # 2. Model (Gemma-2b)
    # Not: Fine-tuned modeli kaydetmediysek orijinali kullanıyoruz.
    model_id = "google/gemma-2b-it"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype=torch.float16
    )

    llm_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=250
    )

    # 3. Vektör Veritabanı
    # (Hız kazanmak için veri setinin tamamını değil, örneği vektörlüyoruz)
    # Gerçek uygulamada bu önceden kaydedilip yüklenir.
    dokumanlar = []
    # Streamlit RAM'i şişmesin diye 3000 satır alıyoruz
    for index, row in df.sample(3000, random_state=42).iterrows():
        metin = (
            f"Başlık: {row['title']}\n"
            f"Yönetmen: {row['director']}\n"
            f"Yıl: {row['release_year']}\n"
            f"Tür: {row['listed_in']}\n"
            f"Özet: {row['description']}"
        )
        dokumanlar.append(metin)

    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vektor_db = Chroma.from_texts(dokumanlar, embedding=embedding_model)

    return df, llm_pipeline, vektor_db, tokenizer

# Yüklemeyi Başlat
df, llm_pipeline, vektor_veritabani, tokenizer = sistemi_yukle()

# --- 2. RAG FONKSİYONU (BEYİN) ---
def rag_yanit(soru):
    # Temizlik Fonksiyonu
    def temizle_yorumlar(metin):
        yasakli = ["I hope", "Please note", "English:", "Turkish:", "Translation:"]
        satirlar = metin.split('\n')
        temiz = []
        for s in satirlar:
            if any(y in s for y in yasakli): break
            if s.strip(): temiz.append(s)
        return " ".join(temiz).strip()

    # Hazırlık
    tur_ceviri = {"korku": "Horror", "gerilim": "Thriller", "aksiyon": "Action", "komedi": "Comed", "drama": "Dram", "bilim kurgu": "Sci-Fi", "aşk": "Romantic"}

    # Soru Temizliği
    temiz_soru = soru.lower()
    for k in ["filmi", "öner", "bana", "yapımı"]:
        temiz_soru = temiz_soru.replace(k, "")
    anahtar = temiz_soru.strip()
    keys = anahtar.split()

    secilen = {}
    yontem = ""

    # A) YIL KONTROLÜ
    yil_match = re.search(r'(\d{4})', soru)
    uygun = pd.DataFrame()

    # Tür bulma
    aranan_tur = None
    for tr, eng in tur_ceviri.items():
        if tr in soru.lower(): aranan_tur = eng

    if yil_match:
        yil = int(yil_match.group(1))
        uygun = df[df['release_year'] == yil]
        yontem = f"YIL ({yil})"
        if aranan_tur and not uygun.empty:
            filt = uygun[uygun['listed_in'].str.contains(aranan_tur, case=False, na=False)]
            if not filt.empty: uygun = filt

    # B) İSİM KONTROLÜ
    elif len(anahtar) > 2:
        temp_c = df.copy()
        temp_d = df.copy()
        for k in keys:
            temp_c = temp_c[temp_c['cast'].str.contains(k, case=False, na=False)]
            temp_d = temp_d[temp_d['director'].str.contains(k, case=False, na=False)]
        uygun = pd.concat([temp_c, temp_d]).drop_duplicates()
        if not uygun.empty: yontem = "İSİM"

    # SEÇİM
    if not uygun.empty:
        row = uygun.sample(1).iloc[0]
        secilen = {"title": row['title'], "year": row['release_year'], "desc": row['description'], "genre": row['listed_in']}
    else:
        # C) VEKTÖR
        if yil_match: return "Aradığınız kriterlere uygun film bulunamadı."
        docs = vektor_veritabani.similarity_search(soru, k=1)
        if docs:
            # Basit parse
            c = docs[0].page_content
            # Burada detaylı parse yerine basitçe contenti alıyoruz
            secilen = {"title": "Vektörden Gelen", "year": "???", "desc": c, "genre": "Konu Bazlı"}

    if not secilen: return "Üzgünüm, sonuç bulamadım."

    # ÇEVİRİ
    if "Başlık:" in secilen.get('desc', ''): # Vektörden geldiyse parse et
        pass # Basitleştirildi

    ozet = secilen.get('desc', 'No desc')
    prompt = f"Translate to Turkish. No comments.\nText: {ozet}\nTranslation:"

    try:
        res = llm_pipeline(prompt, return_full_text=False, max_new_tokens=100)[0]["generated_text"]
        tr_ozet = temizle_yorumlar(res)
    except:
        tr_ozet = ozet

    if secilen['title'] == "Vektörden Gelen":
        return f"Bulunan İçerik:\n{tr_ozet}"

    return f"🎬 **Önerim:** {secilen['title']} ({secilen['year']})\n📂 **Tür:** {secilen['genre']}\n📝 **Konusu:** {tr_ozet}"

# --- 3. CHAT ARAYÜZÜ ---
if "messages" not in st.session_state:
    st.session_state.messages = []

# Geçmiş mesajları ekrana bas
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Kullanıcı girişi
if prompt := st.chat_input("Hangi filmi arıyorsun? (Örn: 2019 korku filmi)"):
    # Kullanıcı mesajını ekle
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)

    # AI Cevabı
    with st.chat_message("assistant"):
        with st.spinner("Film arşivine bakılıyor..."):
            response = rag_yanit(prompt)
            st.markdown(response)

    st.session_state.messages.append({"role": "assistant", "content": response})

In [ ]:
!curl ipv4.icanhazip.com


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501


In [ ]:
!pip install -q pyngrok

In [ ]:
from pyngrok import ngrok
from google.colab import userdata # Colab'in sır kasasını okumak için

# --- GÜVENLİ BAŞLATMA KODU (COLAB SECRETS) ---

try:
    # 1. ADIM: Token'ı güvenli kasadan çek
    # userdata.get() fonksiyonu, verdiğin isimdeki sırrı getirir.
    authtoken = userdata.get('NGROK_AUTH_TOKEN')

    # 2. ADIM: ngrok'u yetkilendir ve tünel aç
    ngrok.set_auth_token(authtoken)
    public_url = ngrok.connect(8501)

    print(f"✅ Web Siten Hazır! Linkin bu: {public_url}")
    print("Not: Modelin yüklenmesi için 2-3 dakika beklemen gerekebilir.")

    # 3. ADIM: Streamlit'i arka planda başlat
    !streamlit run app.py &

except Exception as e:
    print(f"❌ HATA: ngrok token'ını Colab Secrets'e eklediğinden emin misin?")
    print(f"Sol menüdeki Anahtar (🔑) ikonuna tıkla, 'NGROK_AUTH_TOKEN' adında bir sır oluştur.")